# `QuantumOps` performance intro

This notebook introduces some of `QuantumOps` with an emphasis on performance aspects.

Tell `PyCall` to use our virtual Python environment

In [1]:
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")

# import all default symbols for interactive demo
using QuantumOps
using QuantumOps: AbstractOp
using BenchmarkTools
import LinearAlgebra
import SparseArrays

# We also import I, X, Y, Z for convenience
using QuantumOps.Paulis: I, X, Y, Z

### Pauli

These, `I, X, Y, Z`, are bound to instances of the `Pauli` type, representing single-qubit operators.

In [2]:
(I, X, Y, Z) == Pauli.((0, 1, 2, 3)) ## The '.' broadcasts over the following elements.

true

Julia has a large number of standard interfaces and functions for numeric and algebraic types. I follow these when possible. For example, `Matrix` is used to construct a dense, heap-allocated matrix from an object. So I defined a method for it.

In [3]:
print(Matrix.(Pauli.(0:3)))

Matrix[[1.0 0.0; 0.0 1.0], [0.0 1.0; 1.0 0.0], ComplexF64[0.0 + 0.0im 0.0 - 1.0im; 0.0 + 1.0im 0.0 + 0.0im], [1.0 0.0; 0.0 -1.0]]

`Pauli` is in this type hierarchy.

In [4]:
Pauli <: AbstractPauli <: AbstractOp

true

Only a very small amount of code depends on the internals of `Pauli`.
Almost everything is coded against `AbstractOp` and `AbstractPauli`. The developer almost never encounters the implementation of `Pauli`, and the user never does.
But, if you want, you can see it this way.

In [5]:
dump(X)

QuantumOps.Paulis.Pauli
  hi: Bool false
  lo: Bool true


The notation `X[i, j]` calls `getindex(X, i, j)` which, for `AbstractPauli`, looks up the elements in stack allocated arrays.
This is faster than indexing into a heap allocated (that is, every-day, dynamic) array:

In [6]:
m = rand(2, 2) # Ordinary `Matrix`
@btime $m[1, 1];  # @btime is like %timeit. The "$" tests how it would perform in a compiled function

  1.816 ns (0 allocations: 0 bytes)


In [7]:
@btime X[1, 1];  ## This includes time to look up what matrix corresponds to `X`

  1.396 ns (0 allocations: 0 bytes)


Some linear algebra has been implemented.

In Julia, multiplying two small matrices is faster than multiplying two numpy matrices is Python. The Python call includes an overhead.
Multiplying `QuantumOps.Paulis.Pauli` by a `Matrix` is even a bit faster because looking up elements of a `Pauli` is faster.

In [8]:
mY = Matrix(Y) # convert Y to a `Matrix`

@btime Y * $m;

  31.619 ns (1 allocation: 128 bytes)


In [9]:
@btime $mY * $m;

  171.371 ns (1 allocation: 128 bytes)


Another example:

In [10]:
@btime LinearAlgebra.eigvals(Z)

  20.455 ns (1 allocation: 80 bytes)


2-element Vector{Float64}:
 -1.0
  1.0

`20`ns is the time required to copy the array of eigenvalues.

### `PauliTerm`

`PauliTerm` represents a tensor product of Pauli operators (or a single one) and keeps track of a coefficient, including a phase.

### Compare `PauliTerm` with qiskit

In [11]:
using PyCall
qi = pyimport("qiskit.quantum_info");

Here, we compare multiplication of two Pauli strings with both libraries. We see how the time scales with string length.

In [12]:
function get_julia_python_terms(n_qubits)
    xj = PauliTerm(rand(Pauli, n_qubits))
    yj = PauliTerm(rand(Pauli, n_qubits))
    xp = qi.random_pauli(n_qubits)
    yp = qi.random_pauli(n_qubits)
    return (xj, yj, xp, yp)
end

n_qubits = 10
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

# `QuantumOps`
@btime $xj * $yj

  50.930 ns (1 allocation: 80 bytes)


10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
IIZYIZYXZX * (1 + 0im)

In [13]:
# qiskit
@btime $xp.compose($yp)   ## @btime gives same times as %timeit in python cli

  17.601 μs (4 allocations: 256 bytes)


PyObject Pauli('-iIZIZYZXIXI')

#### For mulitplying 10-qubit Pauli strings, `QuantumOps` is about 300 times faster than qiskit.

In [14]:
julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

347.37228190718264

Asymptotically, qiskit is about three times faster than `QuantumOps`. But, it takes a while to get there. For 1000-qubit strings `QuantumOps` is still eight times faster. I have some ideas regarding why python is faster than Julia here, but I am not at all sure. Also, there is a big, ~12 micro-s constant term in the python times. It might be worth trying to reduce this.

In [15]:
n_qubits = 1000
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

9.98148347507805

Here are $10^4$ qubits. Julia is still faster, but they are comparable.

In [16]:
n_qubits = 10^4
(xj, yj, xp, yp) = get_julia_python_terms(n_qubits)

julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp)

qiskit_time / julia_time

1.8758188283735726

### `PauliSum`

A `PauliSum` represents a sum of `PauliTerm`s, sorted in a canonical order.

In [17]:
n_qubits = 10
n_terms = 10
ps = PauliSum(rand(Pauli, (n_terms, n_qubits)), randn(n_terms))

10x10 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{Float64}}:
IIXYIZIXYI * 2.2215889323336255
IYXXYYYYXZ * -0.38178817654468283
XXYYIZXYXZ * -2.039460991887577
YIXIYIZZZZ * -0.5751862981192438
YIYIXZYXYI * -0.18654955526856118
YIZYXZXYZI * 0.5093119169845572
YZZIZXYIXY * 1.0572243529904528
ZIXIZIZIZX * -0.6712398494276668
ZXZIIIIYIY * -0.6472150447189541
ZYIZYZXZXX * -0.2459676130460791

In [18]:
x = ps[5]
(x, typeof(x))

(10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64}:
YIYIXZYXYI * -0.18654955526856118, QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64})

`add!` adds a `PauliTerm` in place. It does a sorted search to find the correct location.

In [19]:
add!(ps, -x)  ## add the additive inverse of a term

9x10 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{Float64}}:
IIXYIZIXYI * 2.2215889323336255
IYXXYYYYXZ * -0.38178817654468283
XXYYIZXYXZ * -2.039460991887577
YIXIYIZZZZ * -0.5751862981192438
YIZYXZXYZI * 0.5093119169845572
YZZIZXYIXY * 1.0572243529904528
ZIXIZIZIZX * -0.6712398494276668
ZXZIIIIYIY * -0.6472150447189541
ZYIZYZXZXX * -0.2459676130460791

The length of the sum is now 9 rather than 10.

In [20]:
length(ps)

9

In [21]:
n_qubits = 10
n_terms = 10
ps = PauliSum(rand(Pauli, (n_terms, n_qubits)), randn(n_terms));
size(ps)

(10, 10)

In [22]:
x = copy(ps[1])

10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Float64}:
IYZYZYYYIX * -1.7648300362371168

In [23]:
@btime add!($ps, $x);

  62.289 ns (0 allocations: 0 bytes)


That seems a bit slow.

### Multiplying Pauli Sums. Compare with Qiskit

`qiskit.quantum_info` and `QuantumOps` are comparable in performance when multiplying Pauli sums;
 at least over some range of parameters.

In [24]:
function get_julia_python_sums(n_qubits, n_terms)
    xj = rand_op_sum(Pauli, n_qubits, n_terms)
    yj = rand_op_sum(Pauli, n_qubits, n_terms)
    xp = qi.SparsePauliOp(qi.random_pauli_list(n_qubits, n_terms))
    yp = qi.SparsePauliOp(qi.random_pauli_list(n_qubits, n_terms))
    return (xj, yj, xp, yp)
end

get_julia_python_sums (generic function with 1 method)

In [25]:
n_qubits = 10; n_terms = 100

(xj, yj, xp, yp) = get_julia_python_sums(n_qubits, n_terms)
julia_time = @belapsed $xj * $xj
qiskit_time = @belapsed $xp.compose($yp).simplify()

qiskit_time / julia_time

1.5243537575018375

In [26]:
n_qubits = 10; n_terms = 2000

(xj, yj, xp, yp) = get_julia_python_sums(n_qubits, n_terms)
julia_time = @elapsed xj * xj
qiskit_time = @elapsed xp.compose(yp).simplify()

qiskit_time / julia_time

0.7094303082828785

In [27]:
n_qubits = 100; n_terms = 2000

(xj, yj, xp, yp) = get_julia_python_sums(n_qubits, n_terms)
julia_time = @elapsed xj * xj
qiskit_time = @elapsed xp.compose(yp).simplify()

qiskit_time / julia_time

1.0960905393336948

#### Pauli decomposition

Construct the Pauli decomposition of a matrix.

In [28]:
m = rand(4, 4)
s = PauliSum(m)

16x2 QuantumOps.PauliSum{Vector{Vector{QuantumOps.Paulis.Pauli}}, Vector{ComplexF64}}:
II * (0.5325669617541431 + 0.0im)
IX * (0.5253614216617813 + 0.0im)
IY * (0.0 + 0.10933946239343692im)
IZ * (0.015554511633424084 + 0.0im)
XI * (0.45449561970641994 + 0.0im)
XX * (0.6144901597409341 + 0.0im)
XY * (0.0 + 0.016330712415224602im)
XZ * (-0.2643962976623996 + 0.0im)
YI * (0.0 - 0.010222264989617286im)
YX * (0.0 + 0.19305786849392814im)
YY * (0.06352046792495009 - 0.0im)
YZ * (0.0 - 0.1528041759924518im)
ZI * (-0.21995259594969138 + 0.0im)
ZX * (0.12543905855302873 + 0.0im)
ZY * (0.0 - 0.24778086294104326im)
ZZ * (0.207191456659944 + 0.0im)

Check that the decomposition is correct.

In [29]:
m ≈ Matrix(s)

true

Doing this decomposition is exponentially expensive. Here we compare the performance of Qiskit QI vs. QuantumOps.

In [30]:
n = 7
m = rand(2^n, 2^n);
julia_time = @belapsed PauliSum($m)

0.082640008

In [31]:
qi_op = qi.Operator(m)
qi_time = @elapsed qi.SparsePauliOp.from_operator(qi_op)

8.774843299

Ratio of times to do Pauli decomposition for random 7-qubit matrix

In [32]:
qi_time / julia_time

106.18153980575607

### Parametric types and composability

#### Z4Group

I implemented a type `Z4Group` that represents `(i, -1, -i, 1)`. This can be used to represent the Pauli group. The type `Z4Group` becomes part of the type of the term, which aids the compiler in devirtualizing and inlining.

In [33]:
t = PauliTerm(:XXY, Z4Group(im))
(t, typeof(t))

(+i XXY, QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, QuantumOps.Z4Groups.Z4Group})

In [34]:
v = PauliTerm(:ZXZ, Z4Group(1))

+1 ZXZ

In [35]:
t * v

+i YIX

#### Z4Group0

More interesting is `Z4Group0` which is `Z4Group` augmented by another `Bool` representing zero. This can represent `(0, im, -im, 1, -1)`. It supports multiplication of elements, but is only closed under addition where at least one operand is `0`. It will error if you don't respect this. This quasi-algebra is enough to represent and compute kronecker products of Pauli matrices. The structure is this

In [36]:
dump(Z4Group0(1))

QuantumOps.Z4Group0s.Z4Group0
  z4: QuantumOps.Z4Groups.Z4Group
    imag: Bool false
    minus: Bool false
  zero: Bool true


Note that this is a nested composite type. Nontheless an array of these is packed, with each element taking three bytes. Here is a packed two-dimensional array of `Z4Group0`.

In [37]:
a = rand(Z4Group0, (3,5))
a

3×5 Matrix{QuantumOps.Z4Group0s.Z4Group0}:
 -i  0   0   -1  -1
 +1  +i  +i  0   +i
 +1  0   0   +1  -1

In [38]:
sizeof(a)  ## (3 x 5) x 3 bytes

45

We see that computation with `Z4Group0` can be as fast as or faster than `Complex{Int}`.

In [39]:
a = rand(Z4Group0, 10^5);
@btime reduce(*, a)

  64.255 μs (1 allocation: 16 bytes)


0

In [40]:
anum = [convert(Complex{Int}, x) for x in a];
@btime reduce(*, anum)

  131.234 μs (1 allocation: 32 bytes)


0 + 0im

I use `Z4Group0` in Kronecker products.

In [41]:
kron([Z4Group0.(m) for m in Matrix.([X, Y, Z])]...)

8×8 Matrix{QuantumOps.Z4Group0s.Z4Group0}:
 0   0   0   0   0   0   -i  0
 0   0   0   0   0   0   0   +i
 0   0   0   0   +i  0   0   0
 0   0   0   0   0   -i  0   0
 0   0   -i  0   0   0   0   0
 0   0   0   +i  0   0   0   0
 +i  0   0   0   0   0   0   0
 0   -i  0   0   0   0   0   0

In [42]:
operators = rand(Pauli, 4)
print(operators)

YXZX

In [43]:
mats = Matrix.(operators)
z40mats = [Z4Group0.(m) for m in mats];

size(kron(mats...))

(16, 16)

In [44]:
@btime kron($mats...);

  729.892 ns (3 allocations: 5.52 KiB)


In [45]:
@btime kron($z40mats...);

  669.594 ns (3 allocations: 1.12 KiB)


Here, the time to do the calcuations with usual 16-byte complex numbers is the same. But, when converting a `PauliSum` to a matrix I use `ThreadsX.sum` over the terms, which is a dropin replacement for `sum` that does intelligent threading. When I use `Z4Group0` I get a significant improvement in performance, perhaps because of fewer cache misses.

### sympy

In [46]:
@pyimport sympy
(x, t) = sympy.symbols("x t")

(PyObject x, PyObject t)

We use a symbolic coefficient

In [47]:
term = PauliTerm("XXYZ", x + t)

4-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, PyObject}:
XXYZ * (PyObject t + x)

In [48]:
term^3

4-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, PyObject}:
XXYZ * (PyObject (t + x)**3)

The type of coefficient is encoded in the type of the `PauliTerm`.

In [49]:
typeof(term)

PauliTerm{Vector{Pauli}, PyObject} (alias for QuantumOps.OpTerm{QuantumOps.Paulis.Pauli, Array{QuantumOps.Paulis.Pauli, 1}, PyObject})

#### `Symbolics`

This is another symbolic libarary.

The following is disabled because of errors due to changes in packages.

using Symbolics
#----------------------------------------------------------------------------

@variables a b c;
#----------------------------------------------------------------------------

# Create a `PauliSum` with symbolic coefficients

term1 = PauliTerm("XZ", a + b)
term2 = PauliTerm("ZX", b + c)

psum = term1^3 + term2
#----------------------------------------------------------------------------

# We convert the `PauliSum` with symbolic coefficients to a `Matrix`.
# No additional code is necessary to support this feature.

symmat = Matrix(psum)
#----------------------------------------------------------------------------

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*